In [22]:
import json

# Load your data (assuming it's in a JSONL format
lang = 'python'
DATA_DIR = 'dataset/'+lang+'/'
data = []
with open(DATA_DIR + "train.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

# Extract code snippets and queries
snippets = [{"code": entry["code_tokens"], "language": lang} for entry in data]
queries = [entry["docstring_tokens"] for entry in data]
code = [entry["code"] for entry in data]

In [23]:
import json

# Load your data
data = []
with open(DATA_DIR + "train.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

# Save tokens to a text file
with open("fasttext_training_data_python.txt", "w") as f:
    for entry in data:
        f.write(" ".join(entry["code_tokens"]) + "\n")
        f.write(" ".join(entry["docstring_tokens"]) + "\n")

In [24]:
import fasttext

# Train a FastText model
model = fasttext.train_unsupervised(
    input="fasttext_training_data_python.txt",  # Path to your training data
    model="skipgram",  # or "cbow"
    dim=100,  # Embedding dimension
    epoch=10,  # Number of epochs
    lr=0.05,  # Learning rate
    minCount=1  # Minimum word frequency
)

# Save the trained model
model.save_model("fasttext_model_python.bin")

Read 30M words
Number of words:  1097380
Number of labels: 1
Progress: 100.0% words/sec/thread:  111704 lr:  0.000000 avg.loss:  0.716834 ETA:   0h 0m 0s  4.7% words/sec/thread:  102678 lr:  0.047633 avg.loss:  1.098448 ETA:   0h 4m46s 73.9% words/sec/thread:  111807 lr:  0.013033 avg.loss:  0.775468 ETA:   0h 1m12s


In [25]:
from codesearch.encoders import BasicEncoder
from functools import partial


from codesearch.text_preprocessing import preprocess_text
from codesearch.code_preprocessing import code_tokenization

# Custom preprocessing parameters
text_preprocessing_params = {"lemmatize": True, "remove_stop": True}
code_preprocessing_params = {"language": lang}

# Initialize the BasicEncoder with custom parameters
encoder = BasicEncoder(
    description_preprocessor=partial(preprocess_text, **text_preprocessing_params),
    code_preprocessor=partial(code_tokenization, **code_preprocessing_params)
)

In [26]:
from codesearch.unif.unif_embedder import UNIFEmbedder
from codesearch.unif.unif_modules import SimilarityModel
from fasttext import load_model

# Load the FastText model
ft_model = load_model("fasttext_model_python.bin")

# Initialize the similarity model
sim_model = SimilarityModel(ft_model)

# Initialize the encoder (you need to define or load this)
# encoder = BasicEncoder() # Replace with your encoder

# Initialize the UNIFEmbedder
unif_embedder = UNIFEmbedder(
    model=sim_model,
    encoder=encoder,
    ft_model=ft_model,
    batch_size=32,  # Adjust based on your hardware
    max_code_len=200,  # Adjust based on your data
    max_description_len=25  # Adjust based on your data
)

Initializing the weights with fast text matrix
Initializing the weights with fast text matrix


In [27]:
queries = [entry["docstring_tokens"] for entry in data]
queries

[['Return',
  'either',
  'the',
  'full',
  'or',
  'truncated',
  'version',
  'of',
  'a',
  'QIIME',
  '-',
  'formatted',
  'taxonomy',
  'string',
  '.'],
 ['Check',
  'to',
  'make',
  'sure',
  'the',
  'supplied',
  'directory',
  'path',
  'does',
  'not',
  'exist',
  'if',
  'so',
  'create',
  'it',
  '.',
  'The',
  'method',
  'catches',
  'OSError',
  'exceptions',
  'and',
  'returns',
  'a',
  'descriptive',
  'message',
  'instead',
  'of',
  're',
  '-',
  'raising',
  'the',
  'error',
  '.'],
 ['Takes',
  'either',
  'a',
  'file',
  'path',
  'or',
  'an',
  'open',
  'file',
  'handle',
  'checks',
  'validity',
  'and',
  'returns',
  'an',
  'open',
  'file',
  'handle',
  'or',
  'raises',
  'an',
  'appropriate',
  'Exception',
  '.'],
 ['Find',
  'the',
  'user',
  'specified',
  'categories',
  'in',
  'the',
  'map',
  'and',
  'create',
  'a',
  'dictionary',
  'to',
  'contain',
  'the',
  'relevant',
  'data',
  'for',
  'each',
  'type',
  'within',
 

In [28]:
# Embed queries
query_embeddings = unif_embedder.embed_queries(queries)

Embedding queries: 100%|██████████| 7870/7870 [00:34<00:00, 227.93it/s]


In [29]:
import numpy as np
# Save query embeddings
np.save("query_embeddings_python.npy", query_embeddings)

In [30]:
# Embed code snippets
snippet_embeddings = unif_embedder.embed_snippets(snippets)

Embedding snippets: 100%|██████████| 7870/7870 [08:54<00:00, 14.74it/s]


In [31]:
# Save code embeddings
np.save("code_embeddings_python.npy", snippet_embeddings) 

In [32]:
unif_embedder.save("unif_embedder_python")

In [33]:
import numpy as np
import json

# Load embeddings
code_embeds = np.load("code_embeddings_python.npy")  # Shape: [num_code_snippets, embed_dim]
query_embeds = np.load("query_embeddings_python.npy")  # Shape: [num_queries, embed_dim]

# Load ground truth (query_id -> correct_code_id)
valid_data = []
lang = 'python'
DATA_DIR = 'dataset/'+lang+'/'
with open(DATA_DIR + "valid.jsonl", "r") as f:
    for line in f:
        valid_data.append(json.loads(line))

In [34]:
# Extract code snippets and queries
valid_snippets = [entry["code_tokens"] for entry in valid_data]
valid_queries = [entry["docstring_tokens"] for entry in valid_data]

In [35]:
import argparse
import logging
import os
import pickle
import random
import torch # type: ignore
import json
import numpy as np
from torch.nn import CrossEntropyLoss, MSELoss # type: ignore
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset # type: ignore

In [36]:
import json
import numpy as np

# Load validation data
with open(DATA_DIR + "valid.jsonl", "r") as f:
    valid_data = [json.loads(line) for line in f]

# Extract queries (docstrings) and codes
queries = [entry["docstring_tokens"] for entry in valid_data]
codes = [entry["code_tokens"] for entry in valid_data]

In [37]:
# Generate query embeddings
query_embeddings = unif_embedder.embed_queries(queries)

# Generate code embeddings
code_snippets = [{"code": code, "language": lang} for code in codes]
code_embeddings = unif_embedder.embed_snippets(code_snippets)

Embedding snippets: 100%|██████████| 435/435 [00:29<00:00, 14.95it/s]


In [38]:
query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
code_embeddings = code_embeddings / np.linalg.norm(code_embeddings, axis=1, keepdims=True)

In [39]:
similarity_matrix = query_embeddings @ code_embeddings.T  # Shape: [num_queries, num_codes]

In [40]:
def evaluate_topk(similarity_matrix, top_k_values=[1, 5, 10]):
    results = {f"Success@{k}": 0 for k in top_k_values}
    reciprocal_ranks = []
    
    for query_idx in range(similarity_matrix.shape[0]):
        # Get scores and sorted indices for this query
        scores = similarity_matrix[query_idx]
        sorted_indices = np.argsort(-scores)  # Descending order
        
        # The correct code is at the same index as the query
        correct_code_idx = query_idx
        rank = np.where(sorted_indices == correct_code_idx)[0][0] + 1  # 1-based
        
        # Update metrics
        for k in top_k_values:
            if rank <= k:
                results[f"Success@{k}"] += 1
        
        reciprocal_ranks.append(1 / rank)
    
    # Normalize results
    num_queries = similarity_matrix.shape[0]
    for k in top_k_values:
        results[f"Success@{k}"] /= num_queries
    results["MRR"] = np.mean(reciprocal_ranks)
    
    return results

In [41]:
# Run evaluation
metrics_python = evaluate_topk(similarity_matrix)
print("Evaluation Results:")
for metric, value in metrics_python.items():
    print(f"{metric}: {value:.4f}")

Evaluation Results:
Success@1: 0.0532
Success@5: 0.1378
Success@10: 0.1957
MRR: 0.1008
